<h1>Dog Breed Identification</h1>

[https://www.kaggle.com/c/dog-breed-identification](https://www.kaggle.com/c/dog-breed-identification/)

Author: Vaasudevan Srinivasan [(Portfolio)](https://vaasudevans.github.io) <br>
Created on: July 06, 2021 <br>

# Import all the Modules

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import tensorflow_hub as hub
import tensorflow as tf
import seaborn as sns
import pandas as pd
import numpy as np

sns.set_theme()
tf.__version__

# Data Preparation

In [ ]:
labels_df = pd.read_csv('/kaggle/input/dog-breed-identification/labels.csv')
labels_df['id'] = labels_df['id'] + '.jpg'
labels_df.breed.value_counts().plot.bar(figsize=(25, 5));

In [ ]:
datagen = ImageDataGenerator(rescale=1.0 / 255,
                             horizontal_flip=True,
                             vertical_flip=True,
                             shear_range=0.2,
                             validation_split=0.1)

train = datagen.flow_from_dataframe(labels_df,
                                    directory='/kaggle/input/dog-breed-identification/train',
                                    x_col='id',
                                    y_col='breed',
                                    subset='training',
                                    target_size=(224, 224),
                                    batch_size=64,
                                    seed=123)

val = datagen.flow_from_dataframe(labels_df,
                                  directory='/kaggle/input/dog-breed-identification/train',
                                  x_col='id',
                                  y_col='breed',
                                  subset='validation',
                                  target_size=(224, 224),
                                  batch_size=64,
                                  seed=123)

In [ ]:
# Get the images from a val batch
imgs = val.next()
labels = list(val.class_indices.keys())

plt.figure(figsize=(20, 6))
cols, rows = 15, 4
for i in range(cols * rows):
    plt.subplot(rows, cols, i + 1)
    plt.imshow(imgs[0][i])
    plt.title(np.argmax(imgs[1][i]))
    plt.axis('off')

plt.tight_layout();

# Transfer Learning using Inception ResNet V2

In [ ]:
url = 'https://tfhub.dev/google/imagenet/inception_resnet_v2/classification/5'
inception_resnet = tf.keras.Sequential([
    hub.KerasLayer(url, trainable=False),
    tf.keras.layers.Dense(120, activation='softmax'),
])
inception_resnet.build((None, 224, 224, 3))
inception_resnet.compile(loss='categorical_crossentropy',
                         optimizer='adam', metrics=['accuracy'])
h = inception_resnet.fit(train, validation_data=val, epochs=10,
                         steps_per_epoch=len(train),
                         validation_steps=len(val))

# Prediction and Submission

In [ ]:
test_df = pd.read_csv('/kaggle/input/dog-breed-identification/sample_submission.csv')
breeds = test_df.columns[1:].tolist()
test_df['file_name'] = test_df['id'] + '.jpg'

In [ ]:
test_gen = ImageDataGenerator(rescale=1.0 / 255)

test = test_gen.flow_from_dataframe(test_df,
                                    x_col='file_name',
                                    y_col=None,
                                    directory='/kaggle/input/dog-breed-identification/test',
                                    class_mode=None,
                                    target=(224, 224))

In [ ]:
# Predict
test_df.loc[:, breeds] = inception_resnet.predict(test)
test_df[breeds].head()

In [ ]:
test_df.drop('file_name', axis=1).to_csv('submission.csv', index=False)